## **A. Load thư viện**

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectFromModel, SequentialFeatureSelector
from sklearn.feature_selection import f_classif

from sklearn.preprocessing import StandardScaler, KBinsDiscretizer

## **B. Một số kỹ thuật cơ bản của Feature selection**

> ### **1. Load dữ liệu ung thư, chọn tập train và test**

In [ ]:
cancer = datasets.load_breast_cancer(as_frame=True)
X_data = cancer.data.iloc[:,0:10]
y_data = cancer.target
display(X_data.head(5))

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=0)
print("Số dữ liệu tập train: %i và tập test: %i" % (X_train.shape[0], X_test.shape[0]))
print("Các class phân loại:", cancer.target_names)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883


Số dữ liệu tập train: 398 và tập test: 171
Các class phân loại: ['malignant' 'benign']


> ### **2. Chọn đặc trưng với Variance Threshold**

In [ ]:
print("Phương sai các đặc trưng:")
display(X_data.var())

## Loại các đặc trưng có phương sai nhỏ hơn 0.01
Feature_sel = VarianceThreshold(threshold=0.01)
Feature_sel.set_output(transform="pandas")
VT_train = Feature_sel.fit_transform(X_train)
VT_test  = X_test[Feature_sel.get_feature_names_out()]

print("\n Các đặc trưng chọn với phương sai lớn hơn 0.01:\n", Feature_sel.get_feature_names_out())
display(VT_train.head(5))

data_train = {'Without FS': X_train, 'With FS': VT_train}
data_test = {'Without FS': X_test, 'With FS': VT_test}
acc, prec, recall = {}, {}, {}
for key in data_train.keys():
    model = linear_model.LogisticRegression()
    model.fit(data_train[key], y_train)
    y_pred = model.predict(data_test[key])
    acc[key] = accuracy_score(y_test, y_pred)
    prec[key] = precision_score(y_test, y_pred)
    recall[key] = recall_score(y_test, y_pred)

df_model = pd.DataFrame(index=data_train.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = acc.values()
df_model['Precision'] = prec.values()
df_model['Recall'] = recall.values()

print("\n Độ chính xác mô hình LogReg giữa chọn và không chọn các đặc trưng:")
display(df_model)

Phương sai các đặc trưng:


mean radius                   12.418920
mean texture                  18.498909
mean perimeter               590.440480
mean area                 123843.554318
mean smoothness                0.000198
mean compactness               0.002789
mean concavity                 0.006355
mean concave points            0.001506
mean symmetry                  0.000752
mean fractal dimension         0.000050
dtype: float64


 Các đặc trưng chọn với phương sai lớn hơn 0.01:
 ['mean radius' 'mean texture' 'mean perimeter' 'mean area']


,mean radius,mean texture,mean perimeter,mean area
478,11.490,14.59,73.99,404.9
303,10.490,18.61,66.86,334.3
155,12.250,17.94,78.27,460.3
186,18.310,18.58,118.60,1041.0
101,6.981,13.43,43.79,143.5



 Độ chính xác mô hình LogReg giữa chọn và không chọn các đặc trưng:


,Accuracy,Precision,Recall
Without FS,0.918129,0.927273,0.944444
With FS,0.918129,0.951923,0.916667


> *Nhận xét: Dựa các hệ số accuracy, precision, recall thì mô hình hồi quy Logistic sau khi loại một số đặc trưng có phương sai nhỏ thì có độ chính xác Precision tốt hơn.*

> ### **3. Chọn đặc trưng với Univariate statistical tests (Kiểm định tương quan các đặc trưng)**

In [ ]:
print("Kiểm định ANOVA")
F_value, P_value = f_classif(X_train, y_train)
Ftest = pd.DataFrame(columns=cancer.feature_names[0:10])
Ftest.loc['F value'] = F_value
Ftest.loc['P value'] = P_value
display(Ftest)

## Chọn các đặc trưng có P-value nhỏ nhất, ít tương quan
Feature_sel = SelectKBest(k=4)
Feature_sel.set_output(transform="pandas")
SKB_train = Feature_sel.fit_transform(X_train, y_train)
SKB_test  = X_test[Feature_sel.get_feature_names_out()]

print("\n Chọn 4 đặc trưng ít tương quan (P value nhỏ nhất):\n", Feature_sel.get_feature_names_out())
display(SKB_train.head(5))

data_train = {'Without FS': X_train, 'With FS': SKB_train}
data_test = {'Without FS': X_test, 'With FS': SKB_test}
acc, prec, recall = {}, {}, {}
for key in data_train.keys():
    model = linear_model.LogisticRegression()
    model.fit(data_train[key], y_train)
    y_pred = model.predict(data_test[key])
    acc[key] = accuracy_score(y_test, y_pred)
    prec[key] = precision_score(y_test, y_pred)
    recall[key] = recall_score(y_test, y_pred)

df_model = pd.DataFrame(index=data_train.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = acc.values()
df_model['Precision'] = prec.values()
df_model['Recall'] = recall.values()

print("\n Độ chính xác mô hình LogReg giữa chọn và không chọn các đặc trưng:")
display(df_model)

Kiểm định ANOVA


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension
F value,4.239174e+02,7.030498e+01,4.566939e+02,3.662638e+02,6.398021e+01,2.467531e+02,4.051114e+02,6.129625e+02,5.041736e+01,0.139085
P value,1.452829e-64,9.009342e-16,6.082261e-68,2.802866e-58,1.399423e-14,1.445033e-43,1.453014e-62,1.935223e-82,5.795974e-12,0.709392



 Chọn 4 đặc trưng ít tương quan (P value nhỏ nhất):
 ['mean radius' 'mean perimeter' 'mean concavity' 'mean concave points']


,mean radius,mean perimeter,mean concavity,mean concave points
478,11.490,73.99,0.05308,0.01969
303,10.490,66.86,0.02297,0.01780
155,12.250,78.27,0.03885,0.02331
186,18.310,118.60,0.08169,0.05814
101,6.981,43.79,0.00000,0.00000



 Độ chính xác mô hình LogReg giữa chọn và không chọn các đặc trưng:


,Accuracy,Precision,Recall
Without FS,0.918129,0.927273,0.944444
With FS,0.894737,0.901786,0.935185


> *Nhận xét: Dựa các hệ số accuracy, precision, recall thì mô hình hồi quy Logistic sau khi chọn một số đặc trưng ít tương quan thì có các độ chính xác không cải thiện.*

> ### **4. Chọn đặc trưng với important features**

In [ ]:
## Chọn các đặc trưng có P-value nhỏ nhất, ít tương quan
Feature_sel = SelectFromModel(estimator=linear_model.LogisticRegression())
Feature_sel.set_output(transform="pandas")
SFM_train = Feature_sel.fit_transform(X_train, y_train)
SFM_test  = X_test[Feature_sel.get_feature_names_out()]

## Hệ số hồi quy mô hình LogReg
print("Hệ số hồi quy LogReg của các đặc trưng:")
coef_value = Feature_sel.estimator_.coef_
EST_coef = pd.DataFrame(columns=cancer.feature_names[0:10])
EST_coef.loc['Coef value'] = coef_value[0]
display(EST_coef)

print("\n Chọn các đặc trưng quan trọng với class (Hệ số hồi quy LogReg lớn):\n", Feature_sel.get_feature_names_out())
display(SFM_train.head(5))

data_train = {'Without FS': X_train, 'With FS': SFM_train}
data_test = {'Without FS': X_test, 'With FS': SFM_test}
acc, prec, recall = {}, {}, {}
for key in data_train.keys():
    model = linear_model.LogisticRegression()
    model.fit(data_train[key], y_train)
    y_pred = model.predict(data_test[key])
    acc[key] = accuracy_score(y_test, y_pred)
    prec[key] = precision_score(y_test, y_pred)
    recall[key] = recall_score(y_test, y_pred)

df_model = pd.DataFrame(index=data_train.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = acc.values()
df_model['Precision'] = prec.values()
df_model['Recall'] = recall.values()

print("\n Độ chính xác mô hình LogReg giữa chọn và không chọn các đặc trưng:")
display(df_model)

Hệ số hồi quy LogReg của các đặc trưng:


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension
Coef value,4.438035,-0.15748,-0.491204,-0.022995,-0.197735,-0.881078,-1.174366,-0.51611,-0.324161,-0.064419



 Chọn các đặc trưng quan trọng với class (Hệ số hồi quy LogReg lớn):
 ['mean radius' 'mean compactness' 'mean concavity']


,mean radius,mean compactness,mean concavity
478,11.490,0.08228,0.05308
303,10.490,0.06678,0.02297
155,12.250,0.06679,0.03885
186,18.310,0.08468,0.08169
101,6.981,0.07568,0.00000



 Độ chính xác mô hình LogReg giữa chọn và không chọn các đặc trưng:


,Accuracy,Precision,Recall
Without FS,0.918129,0.927273,0.944444
With FS,0.918129,0.927273,0.944444


> *Nhận xét: Dựa các hệ số accuracy, precision, recall thì mô hình hồi quy Logistic sau khi chọn một số đặc trưng quan trọng (theo hệ số hồi quy LogReg) thì không cái thiện mô hình.*

> ### **5. Chọn đặc trưng với Sequential Feature Selection**

In [ ]:
## Chọn các đặc trưng vào lần lượt sao cho mô hình có độ chính xác được cải thiện
Feature_sel = SequentialFeatureSelector(estimator=linear_model.Perceptron(), n_features_to_select=4, direction='forward', scoring='accuracy')
Feature_sel.set_output(transform="pandas")
SFS_train = Feature_sel.fit_transform(X_train, y_train)
SFS_test  = X_test[Feature_sel.get_feature_names_out()]

print("\n Chọn các đặc trưng vào mô hình lần lượt:\n", Feature_sel.get_feature_names_out())
display(SFS_train.head(5))

data_train = {'Without FS': X_train, 'With FS': SFS_train}
data_test = {'Without FS': X_test, 'With FS': SFS_test}
acc, prec, recall = {}, {}, {}
for key in data_train.keys():
    model = linear_model.LogisticRegression()
    model.fit(data_train[key], y_train)
    y_pred = model.predict(data_test[key])
    acc[key] = accuracy_score(y_test, y_pred)
    prec[key] = precision_score(y_test, y_pred)
    recall[key] = recall_score(y_test, y_pred)

df_model = pd.DataFrame(index=data_train.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = acc.values()
df_model['Precision'] = prec.values()
df_model['Recall'] = recall.values()

print("\n Độ chính xác mô hình LogReg giữa chọn và không chọn các đặc trưng:")
display(df_model)


 Chọn các đặc trưng vào mô hình lần lượt:
 ['mean smoothness' 'mean compactness' 'mean concavity'
 'mean fractal dimension']


,mean smoothness,mean compactness,mean concavity,mean fractal dimension
478,0.10460,0.08228,0.05308,0.06574
303,0.10680,0.06678,0.02297,0.06600
155,0.08654,0.06679,0.03885,0.06228
186,0.08588,0.08468,0.08169,0.05425
101,0.11700,0.07568,0.00000,0.07818



 Độ chính xác mô hình LogReg giữa chọn và không chọn các đặc trưng:


,Accuracy,Precision,Recall
Without FS,0.918129,0.927273,0.944444
With FS,0.783626,0.759124,0.962963


> *Nhận xét: Dựa các hệ số accuracy, precision, recall thì mô hình hồi quy Logistic sau khi chọn một số đặc trưng lần lượt vào mô hình theo độ chính xác của estimator Perceptron thì không cái thiện mô hình.*

## **C. Một số kỹ thuật cơ bản của Feature transformation**

> ### **1. Chuẩn hóa dữ liệu các đặc trưng với Standard Scaler**

In [ ]:
Feature_trans = StandardScaler()
Feature_trans.set_output(transform="pandas")
SS_train = Feature_trans.fit_transform(X_train)
SS_test  = Feature_trans.transform(X_test)

print("\n Dữ liệu các đặc trưng sau khi chuẩn hóa:")
display(SS_train.head(5))
display('Trung bình', SS_train.mean())
display('Phương sai', SS_train.var())

data_train = {'Without FTrans': X_train, 'With FTrans': SS_train}
data_test = {'Without FTrans': X_test, 'With FTrans': SS_test}
acc, prec, recall = {}, {}, {}
for key in data_train.keys():
    model = linear_model.LogisticRegression()
    model.fit(data_train[key], y_train)
    y_pred = model.predict(data_test[key])
    acc[key] = accuracy_score(y_test, y_pred)
    prec[key] = precision_score(y_test, y_pred)
    recall[key] = recall_score(y_test, y_pred)

df_model = pd.DataFrame(index=data_train.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = acc.values()
df_model['Precision'] = prec.values()
df_model['Recall'] = recall.values()

print("\n Độ chính xác mô hình LogReg giữa chuẩn hóa và không chuẩn hóa các đặc trưng:")
display(df_model)


 Dữ liệu các đặc trưng sau khi chuẩn hóa:


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension
478,-0.749980,-1.099787,-0.741586,-0.701887,0.584593,-0.427726,-0.457550,-0.760550,-0.099860,0.451444
303,-1.028214,-0.139262,-1.029804,-0.894732,0.742882,-0.731843,-0.843301,-0.808805,-1.159759,0.489386
155,-0.538522,-0.299349,-0.568574,-0.550561,-0.714814,-0.731647,-0.639856,-0.668125,0.581758,-0.053477
186,1.147577,-0.146430,1.061698,1.035628,-0.762301,-0.380637,-0.091016,0.221141,-0.663712,-1.225303
101,-2.004538,-1.376954,-1.962370,-1.415904,1.476766,-0.557221,-1.137578,-1.263267,0.439011,2.266825


'Trung bình'

mean radius               1.919179e-16
mean texture              7.051869e-16
mean perimeter            8.926416e-17
mean area                -1.115802e-17
mean smoothness          -3.749095e-16
mean compactness          1.696019e-16
mean concavity            7.141133e-17
mean concave points      -1.115802e-16
mean symmetry             1.467280e-16
mean fractal dimension    3.436670e-16
dtype: float64

'Phương sai'

mean radius               1.002519
mean texture              1.002519
mean perimeter            1.002519
mean area                 1.002519
mean smoothness           1.002519
mean compactness          1.002519
mean concavity            1.002519
mean concave points       1.002519
mean symmetry             1.002519
mean fractal dimension    1.002519
dtype: float64


 Độ chính xác mô hình LogReg giữa chuẩn hóa và không chuẩn hóa các đặc trưng:


,Accuracy,Precision,Recall
Without FTrans,0.918129,0.927273,0.944444
With FTrans,0.941520,0.937500,0.972222


> ### **2. Rời rạc hóa một số đặc trưng định lượng**

In [ ]:
Feature_trans = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
Feature_trans.set_output(transform="pandas")
KBD_train = Feature_trans.fit_transform(X_train)
KBD_test  = Feature_trans.transform(X_test)

print("\n Dữ liệu các đặc trưng sau khi rời rạc hóa:")
display(KBD_train.head(5))

data_train = {'Without FTrans': X_train, 'With FTrans': KBD_train}
data_test = {'Without FTrans': X_test, 'With FTrans': KBD_test}
acc, prec, recall = {}, {}, {}
for key in data_train.keys():
    model = linear_model.LogisticRegression()
    model.fit(data_train[key], y_train)
    y_pred = model.predict(data_test[key])
    acc[key] = accuracy_score(y_test, y_pred)
    prec[key] = precision_score(y_test, y_pred)
    recall[key] = recall_score(y_test, y_pred)

df_model = pd.DataFrame(index=data_train.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = acc.values()
df_model['Precision'] = prec.values()
df_model['Recall'] = recall.values()

print("\n Độ chính xác mô hình LogReg giữa rời rạc hóa và không rời rạc hóa các đặc trưng:")
display(df_model)


 Dữ liệu các đặc trưng sau khi rời rạc hóa:


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension
478,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
303,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
155,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
186,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0



 Độ chính xác mô hình LogReg giữa rời rạc hóa và không rời rạc hóa các đặc trưng:


,Accuracy,Precision,Recall
Without FTrans,0.918129,0.927273,0.944444
With FTrans,0.894737,0.887931,0.953704
